In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import box
import os

# Chemins des fichiers
shp_path = "../../data/project/FORMATION_VEGETALE.shp"  # Shapefile d'entrée
emprise_shp_path = "../../data/project/emprise_etude.shp"  # Shapefile d'emprise
output_path = "../results/data/sample/Sample_BD_foret_T31TCJ.shp"  # Shapefile de sortie

# Charger le shapefile d'entrée
gdf = gpd.read_file(shp_path)

# Charger le shapefile d'emprise
emprise_gdf = gpd.read_file(emprise_shp_path)

# Attributs TFV à traiter et leurs correspondances
mapping = {
    "Forêt fermée d’un autre feuillu pur": ("Autres Feuillus", 11, "Autres feuillus", 11),
    "Forêt fermée de châtaignier pur": ("Autres Feuillus", 11, "Autres feuillus", 11),
    "Forêt fermée de hêtre pur": ("Autres Feuillus", 11, "Autres feuillus", 11),
    "Forêt fermée de chênes décidus purs": ("Chêne", 12, "Chêne", 12),
    "Forêt fermée de robinier pur": ("Robinier", 13, "Robinier", 13),
    "Peupleraie": ("Peupleraie", 14, "Peupleraie", 14),
    "Forêt fermée à mélange de feuillus": (None, None, "Mélange de feuillus", 15),
    "Forêt fermée de feuillus purs en îlots": (None, None, "feuillus en îlots", 16),
    "Forêt fermée d’un autre conifère pur autre que pin": ("Autres conifères autre que pin", 21, "Autres conifères autre que pin", 21),
    "Forêt fermée de mélèze pur": ("Autres conifères autre que pin", 21, "Autres conifères autre que pin", 21),
    "Forêt fermée de sapin ou épicéa": ("Autres conifères autre que pin", 21, "Autres conifères autre que pin", 21),
    "Forêt fermée à mélange d’autres conifères": ("Autres conifères autre que pin", 21, "Autres conifères autre que pin", 21),
    "Forêt fermée d’un autre pin pur": ("Autres pin", 22, "Autres pin", 22),
    "Forêt fermée de pin sylvestre pur": ("Autres pin", 22, "Autres pin", 22),
    "Forêt fermée à mélange de pins purs": ("Autres pin", 22, "Autres pin", 22),
    "Forêt fermée de douglas pur": ("Douglas", 23, "Douglas", 23),
    "Forêt fermée de pin laricio ou pin noir pur": ("Pin laricio ou pin noir", 24, "Pin laricio ou pin noir", 24),
    "Forêt fermée de pin maritime pur": ("Pin maritime", 25, "Pin maritime", 25),
    "Forêt fermée à mélange de conifères": (None, None, "Mélange conifères", 26),
    "Forêt fermée de conifères en îlots": (None, None, "Conifères en îlots", 27),
    "Forêt fermée à mélange de conifères prépondérants et feuillus": (None, None, "Mélange de conifères prépondérants et feuillus", 28),
    "Forêt fermée à mélange de feuillus prépondérants et conifères": (None, None, "Mélange de feuillus prépondérants et conifères", 29),
}

In [2]:
# Ajouter les nouveaux champs "Nom" et "Code" (2 fois pour pixel et objet)
def map_tfv(tfv):
    return mapping.get(tfv, (None, None, None, None))  # Toujours retourner 4 éléments

# Vérifiez que la colonne "TFV" existe
if "TFV" in gdf.columns:
    # Appliquer le mappage et créer les colonnes
    mapped_values = gdf["TFV"].apply(map_tfv)
    mapped_df = pd.DataFrame(mapped_values.tolist(), columns=["Nom_pixel", "Code_pixel", "Nom_objet", "Code_objet"])
    
    # Ajouter les colonnes mappées au GeoDataFrame
    gdf = pd.concat([gdf, mapped_df], axis=1)
    
    # Supprimer les objets dont la valeur de "TFV" n'est pas dans le dictionnaire mapping
    gdf = gdf[gdf["TFV"].isin(mapping.keys())]
    
    print("Les colonnes 'Nom_pixel', 'Code_pixel', 'Nom_objet', et 'Code_objet' ont été créées avec succès.")
else:
    raise ValueError("Erreur : La colonne 'TFV' est absente du GeoDataFrame. Impossible de créer les nouvelles colonnes.")

# Sauvegarder le fichier shapefile temporaire
gdf.to_file(output_path, driver="ESRI Shapefile")
print(f"Fichier shapefile temporaire créé : {output_path}")

Les colonnes 'Nom_pixel', 'Code_pixel', 'Nom_objet', et 'Code_objet' ont été créées avec succès.
Fichier shapefile temporaire créé : ../results/data/sample/Sample_BD_foret_T31TCJ.shp


In [3]:
# Reprojeter le shapefile d'emprise pour correspondre au système de coordonnées du raster
emprise_gdf = emprise_gdf.to_crs(gdf.crs)

# Découper le shapefile d'entrée en utilisant l'emprise du shapefile
gdf_clipped = gdf[gdf.geometry.intersects(emprise_gdf.geometry.unary_union)]

# Sauvegarder le fichier shapefile final
gdf_clipped.to_file(output_path, driver="ESRI Shapefile")
print(f"Shapefile découpé créé : {output_path}")

/tmp/ipykernel_376405/2104086327.py:5: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  gdf_clipped = gdf[gdf.geometry.intersects(emprise_gdf.geometry.unary_union)]


Shapefile découpé créé : ../results/data/sample/Sample_BD_foret_T31TCJ.shp
